In [1]:
# ============================================
# IMPORTAR LIBRERÍAS
# ============================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# ============================================
# CARGA DE DATOS
# ============================================
archivo = "pacientes_100_final.csv"  # Cambia si tu archivo tiene otro nombre
df = pd.read_csv(archivo)

print(f"\nDatos cargados correctamente desde '{archivo}'")
print(f"Total de registros: {len(df)}")
print(f"Columnas disponibles: {df.columns.tolist()}")
print("\nPrimeras filas:")
print(df.head())

# ============================================
# LIMPIEZA Y MAPEOS
# ============================================

# Estandarizar nombres de columnas a mayúscula inicial
df.columns = [c.strip().capitalize() for c in df.columns]

# Renombrar columnas específicas si es necesario
renombres = {
    'Edad': 'Edad',
    'Sexo': 'Sexo',
    'Ejercicio': 'Ejercicio',
    'Alcohol': 'Alcohol',
    'Menstruacion': 'Menstruacion',
    'Primerhijo': 'PrimerHijo',
    'Mamografia': 'Mamografia',
    'Familiarprimergradocc': 'FamiliarPrimerGradoCC',
    'Familiarsegundogradocc': 'FamiliarSegundoGradoCC',
    'Diagnosticopreviocancer': 'DiagnosticoPrevioCancer'
}
df.rename(columns=renombres, inplace=True)

print(f"\nColumnas después de estandarizar: {df.columns.tolist()}")

# Categorizar Edad
def categorizar_edad(edad):
    try:
        edad = float(edad)
        if edad < 40: return 0
        elif edad < 50: return 1
        elif edad < 60: return 2
        else: return 3
    except:
        return np.nan

if 'Edad' in df.columns:
    df['Edad'] = df['Edad'].apply(categorizar_edad)
else:
    print("No se encontró columna 'Edad'")

# Mapear texto a valores numéricos
mapeos = {
    'Sexo': {'F': 0, 'M': 1},
    'Menstruacion': {'<12 años': 2, '12-13 años': 1, '>=14 años': 0, '12–13 años': 1, '>=14 anos': 0},
    'PrimerHijo': {'No ha tenido hijos': 2, 'No hijos': 2, 'menor a 30 años': 1, 'mayor a 30 años': 0},
    'Ejercicio': {'Ninguna': 0, '<3 horas': 1, '3-4 horas': 2, '>4 horas': 3, '3–4 horas': 2},
    'Alcohol': {'Nunca': 0, 'Ocasional': 1, 'Frecuente': 2, 'Diario': 3}
}

for col, mapping in mapeos.items():
    if col in df.columns:
        df[col] = df[col].map(mapping)
    else:
        print(f"Columna '{col}' no encontrada (se omitirá del modelo)")

# Booleanas ("Si"/"No" → 1/0)
booleanas = ['FamiliarPrimerGradoCC', 'FamiliarSegundoGradoCC', 'DiagnosticoPrevioCancer', 'Mamografia']

for col in booleanas:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({'si': 1, 'sí': 1, 'no': 0, '0': 0, '1': 1})
        df[col] = df[col].astype(int)
    else:
        print(f"Columna '{col}' no encontrada (se omitirá del modelo)")

# Eliminar filas con valores nulos
df = df.dropna()
print(f"\nRegistros después de limpieza: {len(df)}")
print(df.head())

# ============================================
# CÁLCULO DEL PUNTAJE Y CLASIFICACIÓN
# ============================================

def calcular_puntaje(fila):
    puntaje = 0

    # Edad
    puntaje += [0, 5, 12, 20][int(fila['Edad'])]

    # Sexo
    puntaje += 5 if fila['Sexo'] == 1 else 0

    # Familiares
    puntaje += 25 if fila['FamiliarPrimerGradoCC'] == 1 else 0
    puntaje += 4 if fila['FamiliarSegundoGradoCC'] == 1 else 0

    # Antecedente personal
    puntaje += 30 if fila['DiagnosticoPrevioCancer'] == 1 else 0

    # Menstruación
    puntaje += [6, 3, 0][int(fila['Menstruacion'])]

    # Primer hijo
    puntaje += [0, 5, 8][int(fila['PrimerHijo'])]

    # Ejercicio
    puntaje += [5, 3, -2, -5][int(fila['Ejercicio'])]

    # Alcohol
    puntaje += [0, 2, 5, 8][int(fila['Alcohol'])]

    # Mamografía
    puntaje += -3 if fila['Mamografia'] == 1 else 0

    return puntaje

df['Puntaje'] = df.apply(calcular_puntaje, axis=1)

def clasificar_riesgo(puntaje):
    if puntaje < 20: return 0  # Bajo
    elif puntaje < 50: return 1  # Moderado
    else: return 2  # Alto

df['Riesgo'] = df['Puntaje'].apply(clasificar_riesgo)

print("\nEjemplo de puntajes y riesgo calculado:")
print(df[['Edad', 'Sexo', 'Puntaje', 'Riesgo']].head())

# ============================================
# PREPARACIÓN DE VARIABLES
# ============================================

X = df[['Edad','Sexo','FamiliarPrimerGradoCC','FamiliarSegundoGradoCC','DiagnosticoPrevioCancer',
        'Menstruacion','PrimerHijo','Ejercicio','Alcohol','Mamografia']]
y = df['Riesgo']

print(f"\nVariables predictoras (X): {X.shape}")
print(f"Variable objetivo (y): {y.shape}")
print("\nDistribución de clases:")
print(y.value_counts())


Datos cargados correctamente desde 'pacientes_100_final.csv'
Total de registros: 100
Columnas disponibles: ['id_paciente', 'nombre', 'rut', 'edad', 'sexo', 'fecha_registro', 'id_medico', 'FamiliarPrimerGradoCC', 'FamiliarSegundoGradoCC', 'DiagnosticoPrevioCancer', 'Ejercicio', 'Alcohol', 'Mamografia', 'Menstruacion', 'PrimerHijo', 'Diagnostico']

Primeras filas:
   id_paciente               nombre         rut  edad sexo fecha_registro  \
0            1  Inmaculada Arellano  20149287-4    53    F     09-10-2025   
1            2      Encarna Naranjo  20451679-1    71    F     09-10-2025   
2            3       Paulina Cortes  20135229-8    69    F     09-10-2025   
3            4           Agata Diaz  18113446-5    58    F     09-10-2025   
4            5           Maura Pena  20512733-6    26    F     09-10-2025   

   id_medico  FamiliarPrimerGradoCC  FamiliarSegundoGradoCC  \
0          1                      1                       1   
1          1                      0          

In [2]:
# ============================================
# PREPARACIÓN DE DATOS PARA ML
# ============================================

columnas_predictoras = [
    'Edad', 'Sexo', 'FamiliarPrimerGradoCC', 'FamiliarSegundoGradoCC',
    'DiagnosticoPrevioCancer', 'Menstruacion', 'PrimerHijo',
    'Ejercicio', 'Alcohol', 'Mamografia'
]

X = df[columnas_predictoras]
y = df['Riesgo']

print(f"\nVariables predictoras: {X.shape}")
print(f"Variable objetivo: {y.shape}")

# División segura
if len(df) < 10:
    X_train, X_test, y_train, y_test = X, X, y, y
    print("\nDataset pequeño: usando todos los datos para entrenamiento y prueba.")
else:
    stratify_param = y if y.value_counts().min() > 1 else None
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=stratify_param
    )


Variables predictoras: (12, 10)
Variable objetivo: (12,)


In [3]:
# ============================================
# MODELO 1: RANDOM FOREST
# ============================================

print("\nEntrenando modelo Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
accuracy_rf = rf.score(X_test, y_test)

print(f"Precisión Random Forest: {accuracy_rf:.2%}")
print("\nImportancia de características:")
print(pd.DataFrame({
    'Característica': X.columns,
    'Importancia': rf.feature_importances_
}).sort_values('Importancia', ascending=False))

# Guardar modelo
with open('modelo_random_forest.pkl', 'wb') as f:
    pickle.dump(rf, f)


Entrenando modelo Random Forest...
Precisión Random Forest: 33.33%

Importancia de características:
            Característica  Importancia
0                     Edad     0.193080
7                Ejercicio     0.187747
9               Mamografia     0.165921
2    FamiliarPrimerGradoCC     0.160067
8                  Alcohol     0.121751
4  DiagnosticoPrevioCancer     0.099593
3   FamiliarSegundoGradoCC     0.071841
1                     Sexo     0.000000
5             Menstruacion     0.000000
6               PrimerHijo     0.000000


In [5]:
# ============================================
# MODELO 2: K-NEAREST NEIGHBORS (KNN)
# ============================================

from sklearn.neighbors import KNeighborsClassifier


k_value = max(3, int(np.sqrt(len(X_train))))
if k_value % 2 == 0:
    k_value += 1

print(f"\nEntrenando modelo KNN con K={k_value}...")
knn = KNeighborsClassifier(n_neighbors=k_value, metric='euclidean', weights='distance')
knn.fit(X_train, y_train)
accuracy_knn = knn.score(X_test, y_test)
print(f"Precisión KNN: {accuracy_knn:.2%}")

with open('modelo_knn.pkl', 'wb') as f:
    pickle.dump(knn, f)


Entrenando modelo KNN con K=3...
Precisión KNN: 66.67%


In [8]:
# ============================================
# MODELO 3: K-MEANS
# ============================================

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score


print("\nEntrenando modelo K-Means (3 clusters)...")
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X)
labels = kmeans.predict(X)

df['Cluster'] = labels
ari = adjusted_rand_score(y, labels)
print(f"Ajuste Rand Index (similitud clusters vs riesgo): {ari:.4f}")

with open('modelo_kmeans.pkl', 'wb') as f:
    pickle.dump(kmeans, f)


Entrenando modelo K-Means (3 clusters)...


c:\Users\benja\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Ajuste Rand Index (similitud clusters vs riesgo): 0.0471


In [9]:
# ============================================
# COMPARACIÓN FINAL
# ============================================

print("\n" + "="*70)
print("COMPARACIÓN FINAL DE MODELOS")
print("="*70)

comparacion = pd.DataFrame({
    'Modelo': ['Random Forest', 'KNN', 'K-Means'],
    'Tipo': ['Supervisado', 'Supervisado', 'No supervisado'],
    'Precisión': [f"{accuracy_rf:.2%}", f"{accuracy_knn:.2%}", f"{ari:.2%}"]
})

print(comparacion.to_string(index=False))
print("\nArchivos generados:")
print("  - modelo_random_forest.pkl")
print("  - modelo_knn.pkl")
print("  - modelo_kmeans.pkl")
print("Proceso completado exitosamente")


COMPARACIÓN FINAL DE MODELOS
       Modelo           Tipo Precisión
Random Forest    Supervisado    33.33%
          KNN    Supervisado    66.67%
      K-Means No supervisado     4.71%

Archivos generados:
  - modelo_random_forest.pkl
  - modelo_knn.pkl
  - modelo_kmeans.pkl
Proceso completado exitosamente


In [10]:
# ============================================
# PREDICCIÓN DE NUEVO PACIENTE CON KNN
# ============================================
import pickle
import pandas as pd

# Cargar el modelo KNN entrenado
with open('modelo_knn.pkl', 'rb') as f:
    knn = pickle.load(f)

print("Modelo KNN cargado correctamente\n")

# ============================================
# INGRESO DE NUEVO PACIENTE
# ============================================

# Ingreso manual de datos
nuevo_paciente = {
    'Edad': int(input("Edad del paciente: ")),
    'Sexo': input("Sexo (F/M): ").strip().upper(),
    'FamiliarPrimerGradoCC': input("Familiar 1er grado con cáncer (Si/No): ").strip().lower(),
    'FamiliarSegundoGradoCC': input("Familiar 2do grado con cáncer (Si/No): ").strip().lower(),
    'DiagnosticoPrevioCancer': input("Diagnóstico previo de cáncer (Si/No): ").strip().lower(),
    'Menstruacion': input("Edad de la primera menstruación (<12 años / 12-13 años / >=14 años): ").strip(),
    'PrimerHijo': input("Primer hijo (No ha tenido hijos / menor a 30 años / mayor a 30 años): ").strip(),
    'Ejercicio': input("Ejercicio semanal (Ninguna / <3 horas / 3-4 horas / >4 horas): ").strip(),
    'Alcohol': input("Consumo de alcohol (Nunca / Ocasional / Frecuente / Diario): ").strip(),
    'Mamografia': input("Se ha realizado mamografía (Si/No): ").strip().lower()
}

# ============================================
# TRANSFORMACIONES (mismo formato que el dataset)
# ============================================

# Categorizar edad
def categorizar_edad(edad):
    if edad < 40: return 0
    elif edad < 50: return 1
    elif edad < 60: return 2
    else: return 3

nuevo_paciente['Edad'] = categorizar_edad(nuevo_paciente['Edad'])

# Mapear valores categóricos
mapeos = {
    'Sexo': {'F': 0, 'M': 1},
    'Menstruacion': {'<12 años': 2, '12-13 años': 1, '>=14 años': 0, '12–13 años': 1, '>=14 anos': 0},
    'PrimerHijo': {'No ha tenido hijos': 2, 'No hijos': 2, 'menor a 30 años': 1, 'mayor a 30 años': 0},
    'Ejercicio': {'Ninguna': 0, '<3 horas': 1, '3-4 horas': 2, '>4 horas': 3, '3–4 horas': 2},
    'Alcohol': {'Nunca': 0, 'Ocasional': 1, 'Frecuente': 2, 'Diario': 3},
}

# Booleanos Si/No → 1/0
booleanas = ['FamiliarPrimerGradoCC','FamiliarSegundoGradoCC','DiagnosticoPrevioCancer','Mamografia']

# Aplicar mapeos
for col, mapping in mapeos.items():
    if nuevo_paciente[col] in mapping:
        nuevo_paciente[col] = mapping[nuevo_paciente[col]]
    else:
        print(f"Valor inesperado en {col}: {nuevo_paciente[col]}")
        nuevo_paciente[col] = 0

# Booleanos
for col in booleanas:
    nuevo_paciente[col] = 1 if nuevo_paciente[col] in ['si','sí','1'] else 0

# ============================================
# PREDICCIÓN
# ============================================

# Convertir a DataFrame
df_nuevo = pd.DataFrame([nuevo_paciente])

# Reordenar columnas según el modelo
columnas_modelo = ['Edad','Sexo','FamiliarPrimerGradoCC','FamiliarSegundoGradoCC','DiagnosticoPrevioCancer',
                   'Menstruacion','PrimerHijo','Ejercicio','Alcohol','Mamografia']
df_nuevo = df_nuevo[columnas_modelo]

# Hacer predicción
prediccion = knn.predict(df_nuevo)[0]

# Mapeo de resultado
riesgo_texto = {0: 'Bajo', 1: 'Moderado', 2: 'Alto'}

print("\n======================================")
print("RESULTADO DEL PACIENTE NUEVO")
print("======================================")
print(df_nuevo.to_string(index=False))
print(f"\nNivel de riesgo estimado: {riesgo_texto[prediccion]}")
print("======================================")

Modelo KNN cargado correctamente


RESULTADO DEL PACIENTE NUEVO
 Edad  Sexo  FamiliarPrimerGradoCC  FamiliarSegundoGradoCC  DiagnosticoPrevioCancer  Menstruacion  PrimerHijo  Ejercicio  Alcohol  Mamografia
    0     1                      1                       0                        0             1           2          1        0           0

Nivel de riesgo estimado: Moderado


In [11]:
# ============================================
# PREDICCIÓN DE NUEVO PACIENTE CON KNN
# ============================================
import pickle
import pandas as pd

# Cargar el modelo Kmeans entrenado
with open('modelo_kmeans.pkl', 'rb') as f:
    knn = pickle.load(f)

print("Modelo Kmeans cargado correctamente\n")

# ============================================
# INGRESO DE NUEVO PACIENTE
# ============================================

# Ingreso manual de datos
nuevo_paciente = {
    'Edad': int(input("Edad del paciente: ")),
    'Sexo': input("Sexo (F/M): ").strip().upper(),
    'FamiliarPrimerGradoCC': input("Familiar 1er grado con cáncer (Si/No): ").strip().lower(),
    'FamiliarSegundoGradoCC': input("Familiar 2do grado con cáncer (Si/No): ").strip().lower(),
    'DiagnosticoPrevioCancer': input("Diagnóstico previo de cáncer (Si/No): ").strip().lower(),
    'Menstruacion': input("Edad de la primera menstruación (<12 años / 12-13 años / >=14 años): ").strip(),
    'PrimerHijo': input("Primer hijo (No ha tenido hijos / menor a 30 años / mayor a 30 años): ").strip(),
    'Ejercicio': input("Ejercicio semanal (Ninguna / <3 horas / 3-4 horas / >4 horas): ").strip(),
    'Alcohol': input("Consumo de alcohol (Nunca / Ocasional / Frecuente / Diario): ").strip(),
    'Mamografia': input("Se ha realizado mamografía (Si/No): ").strip().lower()
}

# ============================================
# TRANSFORMACIONES (mismo formato que el dataset)
# ============================================

# Categorizar edad
def categorizar_edad(edad):
    if edad < 40: return 0
    elif edad < 50: return 1
    elif edad < 60: return 2
    else: return 3

nuevo_paciente['Edad'] = categorizar_edad(nuevo_paciente['Edad'])

# Mapear valores categóricos
mapeos = {
    'Sexo': {'F': 0, 'M': 1},
    'Menstruacion': {'<12 años': 2, '12-13 años': 1, '>=14 años': 0, '12–13 años': 1, '>=14 anos': 0},
    'PrimerHijo': {'No ha tenido hijos': 2, 'No hijos': 2, 'menor a 30 años': 1, 'mayor a 30 años': 0},
    'Ejercicio': {'Ninguna': 0, '<3 horas': 1, '3-4 horas': 2, '>4 horas': 3, '3–4 horas': 2},
    'Alcohol': {'Nunca': 0, 'Ocasional': 1, 'Frecuente': 2, 'Diario': 3},
}

# Booleanos Si/No → 1/0
booleanas = ['FamiliarPrimerGradoCC','FamiliarSegundoGradoCC','DiagnosticoPrevioCancer','Mamografia']

# Aplicar mapeos
for col, mapping in mapeos.items():
    if nuevo_paciente[col] in mapping:
        nuevo_paciente[col] = mapping[nuevo_paciente[col]]
    else:
        print(f"Valor inesperado en {col}: {nuevo_paciente[col]}")
        nuevo_paciente[col] = 0

# Booleanos
for col in booleanas:
    nuevo_paciente[col] = 1 if nuevo_paciente[col] in ['si','sí','1'] else 0

# ============================================
# PREDICCIÓN
# ============================================

# Convertir a DataFrame
df_nuevo = pd.DataFrame([nuevo_paciente])

# Reordenar columnas según el modelo
columnas_modelo = ['Edad','Sexo','FamiliarPrimerGradoCC','FamiliarSegundoGradoCC','DiagnosticoPrevioCancer',
                   'Menstruacion','PrimerHijo','Ejercicio','Alcohol','Mamografia']
df_nuevo = df_nuevo[columnas_modelo]

# Hacer predicción
prediccion = knn.predict(df_nuevo)[0]

# Mapeo de resultado
riesgo_texto = {0: 'Bajo', 1: 'Moderado', 2: 'Alto'}

print("\n======================================")
print("RESULTADO DEL PACIENTE NUEVO")
print("======================================")
print(df_nuevo.to_string(index=False))
print(f"\nNivel de riesgo estimado: {riesgo_texto[prediccion]}")
print("======================================")

Modelo Kmeans cargado correctamente

Valor inesperado en Menstruacion: >=14

RESULTADO DEL PACIENTE NUEVO
 Edad  Sexo  FamiliarPrimerGradoCC  FamiliarSegundoGradoCC  DiagnosticoPrevioCancer  Menstruacion  PrimerHijo  Ejercicio  Alcohol  Mamografia
    0     0                      1                       1                        1             0           1          0        3           0

Nivel de riesgo estimado: Bajo
